# 第9章: ベクトル空間法 (I)

enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

In [37]:
import numpy as np

# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
+ 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [23]:
with open('enwiki-20150112-400-r100-10576.txt', 'r') as f:
    delete_words = ".,!?;:()[]'" + '"'
    corpus = []
    for line in f.readlines():
        for token in line[:-1].split():
            if token[0] in delete_words:
                token = token[1:]
            if len(token) == 0:
                break
            if token[-1] in delete_words:
                token = token[:-1]
            if token:
                corpus.append(token)
with open('corpus.txt', 'w') as f:
    f.write(" ".join(corpus))

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [23]:
# http://www.projectvisa.com/fullcountrylist.asp
with open('country.txt', 'r') as f:
    countries = [x[:-1] for x in f.readlines() if " " in x]

with open('corpus.txt', 'r') as f:
    corpus = f.read()[:-1]

In [24]:
for c in countries:
    corpus = corpus.replace(c, c.replace(" ", "_"))

In [79]:
corpus_list = corpus.split(" ")
dictionary = np.array(list(set(corpus_list)))
T = len(corpus_list)

# 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

+ ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
+ 単語$t$を選ぶ度に，文脈幅$d$は$\{1,2,3,4,5\}$の範囲でランダムに決める．

In [81]:
def get_d():
    d = np.random.randint(-5, 5)
    if d == 0:
        d = get_d()
    return d

with open("82.txt", "w") as f:
    for i, t in enumerate(corpus_list):
        d = get_d() 
        if (i + d >= 0) and  (i + d < T):
            c = corpus_list[i + d]
            if c != t:
                line = t + "\t" + c + "\n"
                f.write(line)

# 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

+ $f(t,c)$: 単語$t$と文脈語$c$の共起回数
+ $f(t,∗)$: 単語$t$の出現回数
+ $f(∗,c)$: 文脈語$c$の出現回数
+ $N$: 単語と文脈語のペアの総出現回数

In [82]:
import pandas as pd
tc = pd.read_csv("82.txt", sep="\t", header=None, names=["t", "c"])
tc = tc[tc.t != tc.c]

In [83]:
N = len(tc.index)
N

11258542

In [84]:
tc.head()

,t,c
0,Anarchism,political
1,Anarchism,political
2,is,that
3,philosophy,Anarchism
4,that,a


In [85]:
def f(t=None, c=None):
    # f(t, c)
    if t is not None and c is not None:
        return len(tc[(tc.t == t) &(tc.c == c)].index)
    # f(t, *)
    if t is not None and c is None:
        # return len(tc[tc.t == t])
        return len(tc[tc.t == t].index)
    # f(*, c)
    if t is None and c is not None:
        return len(tc[tc.c == c].index)

In [86]:
time len(tc[tc.t == 'a'])

CPU times: user 1.61 s, sys: 32.6 ms, total: 1.64 s
Wall time: 1.72 s


221305

In [87]:
time len(tc[tc.t == 'a'].index)

CPU times: user 1.37 s, sys: 16.3 ms, total: 1.38 s
Wall time: 1.38 s


221305

In [88]:
time f('a', None)

CPU times: user 1.53 s, sys: 25.2 ms, total: 1.56 s
Wall time: 1.6 s


221305

In [34]:
f('Stromberg', 'Stromberg')

0

# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

+ $f(t,c)≥10$ならば，$X_{tc}=PPMI(t,c)=max\{log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
+ $f(t,c)<10$ならば，$X_{tc}=0$

ここで，$PPMI(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．
なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [40]:
from scipy import sparse
W = len(dictionary)
X = sparse.lil_matrix((W, W))

In [45]:
dictionary[100]

'pressured'

In [90]:
for t_i in range(W):
    print(t_i)
    for c_i in range(W):
        t = dictionary[t_i]
        c = dictionary[c_i]
        ftc = f(t, c)
        if ftc >= 10:
            ft = f(t, None)
            fc = f(None, c)
            ppmi = log((N * ftc) / (ft * fc))
            if ppmi > 0:
                X[t_i, c_i] = ppmi

0


KeyboardInterrupt: 

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

# 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

# 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

# 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

# 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ